In [1]:
# Import libraries
import tarfile
import scanpy as sc
import os
import anndata as ad

In [8]:
# For loop to create an adata for the cancer dataset
# Extract files to a directory
data_dir = "/Users/saidalkildani/Desktop/DS_AI/Projects/EndoGyn/data/raw/cancer"
with tarfile.open("data/raw/cancer/GSE184880_RAW.tar", "r") as tar:
    tar.extractall(path="data/raw/cancer")
# List all prefixes (one per sample)
prefixes = [
    "GSM5599220_Norm1",
    "GSM5599221_Norm2",
    "GSM5599222_Norm3",
    "GSM5599223_Norm4",
    "GSM5599224_Norm5",
    "GSM5599225_Cancer1",
    "GSM5599226_Cancer2",
    "GSM5599227_Cancer3",
    "GSM5599228_Cancer4",
    "GSM5599229_Cancer5",
    "GSM5599230_Cancer6",
    "GSM5599231_Cancer7"
]

adatas = []
for prefix in prefixes:
    # Rename files to match Scanpy's expectations

    os.rename(
        os.path.join(data_dir, f"{prefix}.matrix.mtx.gz"),
        os.path.join(data_dir, f"{prefix}_matrix.mtx.gz")
    )
    os.rename(
        os.path.join(data_dir, f"{prefix}.barcodes.tsv.gz"),
        os.path.join(data_dir, f"{prefix}_barcodes.tsv.gz")
    )
    os.rename(
        os.path.join(data_dir, f"{prefix}.genes.tsv.gz"),
        os.path.join(data_dir, f"{prefix}_features.tsv.gz")
    )
    prefix = prefix + "_" # add underscore 
    adata = sc.read_10x_mtx(
        data_dir,
        prefix=prefix,
        var_names='gene_symbols',
        make_unique=True,
        cache=True
    )
    adata.obs["sample"] = prefix.rstrip("_")
    adatas.append(adata)
# Concatenate all Objects inside adatas into one
adata_cancer = ad.concat(adatas, label="batch", index_unique="-", keys=[p.rstrip("_") for p in prefixes])

In [9]:
# Extract files to a directory
data_dir = "/Users/saidalkildani/Desktop/DS_AI/Projects/EndoGyn/data/raw/endo"
with tarfile.open("data/raw/endo/GSE214411_RAW.tar", "r") as tar:
    tar.extractall(path="data/raw/endo")
# List all prefixes (one per sample)
prefixes = [
    "GSM6605431_EMS1_",
    "GSM6605432_EMS2_",
    "GSM6605433_EMS3_",
    "GSM6605434_EMS4_",
    "GSM6605435_EMS5_",
    "GSM6605436_EMS6_",
    "GSM6605437_N1_",
    "GSM6605438_N2_",
    "GSM6605439_N3_",
    "GSM6605440_N4_",
    "GSM7277296_N-5_",
    "GSM7277297_N-6_",
    "GSM7277298_N-7_"
]

adatas = []
for prefix in prefixes:
    adata = sc.read_10x_mtx(
        data_dir,
        prefix=prefix,
        var_names='gene_symbols',
        make_unique=True,
        cache=True
    )
    adata.obs["sample"] = prefix.rstrip("_")
    adatas.append(adata)
# Concatenate all Objects inside adatas into one
adata_endo = ad.concat(adatas, label="batch", index_unique="-", keys=[p.rstrip("_") for p in prefixes])

In [10]:
def preprocess(adata):
    # Basic filtering
    sc.pp.filter_cells(adata, min_genes=200)
    sc.pp.filter_genes(adata, min_cells=3)
    # Annotate mito genes and filter them 
    adata.var["mt"] = adata.var_names.str.startswith("MT-")
    sc.pp.calculate_qc_metrics(
    adata, qc_vars=["mt"], percent_top=None, log1p=False, inplace=True
    )
    adata = adata[adata.obs.n_genes_by_counts < 2500, :]
    adata = adata[adata.obs.pct_counts_mt < 20, :].copy()
    # Normalize
    sc.pp.normalize_total(adata, target_sum=1e4)
    # Logarithmize
    sc.pp.log1p(adata)
    # Identify highly variable genes
    sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5)
    adata = adata[:, adata.var.highly_variable]
    return adata

In [ ]:
def vis_umap(adata, x):
    # Scale
    sc.pp.scale(adata, max_value=10)    
    # PCA
    sc.tl.pca(adata, svd_solver="arpack")
    sc.pp.neighbors(adata, n_neighbors=10, n_pcs=40)
    sc.tl.leiden(
        adata,
        resolution=0.25,
        random_state=0
    )
    return sc.tl.umap(adata, color= x)

In [11]:
# Highly variable genes
adata_cancer = preprocess(adata_cancer)
adata_endo = preprocess(adata_endo)

In [12]:
# Merge the two datasets
merged_adata = ad.concat(
    [adata_endo, adata_cancer],
    axis=0,          # Concatenate along cells 
    join="outer",    # Keep all genes only
    keys=["Endometriosis", "Cancer"],
    merge="unique",  # Handle overlapping metadata uniquely
    index_unique="-",  # Avoid duplicate observation names
    fill_value=0     # Fill missing values wth zeros
)

In [13]:
# Assign datasets as a column
merged_adata.obs['dataset'] = merged_adata.obs.index.str.split('-').str[-1]

In [14]:
# Create target variable in obs
merged_adata.obs["target"] = (
    merged_adata.obs["sample"]
    .str.split("_").str[-1]
    .str[0]
    .map({"C": "Cancer", "E": "EMS", "N": "Normal"})
)

# scVI Integration

In [15]:
from sklearn.model_selection import train_test_split

In [ ]:
# Stratified split preserving target distribution
train_idx, test_idx = train_test_split(
    merged_adata.obs.index,
    test_size=0.2,
    stratify=merged_adata.obs["target"],
    random_state=42
)

In [17]:
adata_train = merged_adata[train_idx].copy()
adata_test = merged_adata[test_idx].copy()

In [21]:
adata_train.layers["counts"] = adata_train.X

In [26]:
adata_test.layers["counts"] = adata_test.X

In [19]:
import scvi

/Users/saidalkildani/Desktop/DS_AI/Projects/EndoGyn/venv1.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Setup model and train
scvi.model.SCVI.setup_anndata(adata_train, layer="counts", 
                              categorical_covariate_keys= ["sample", "dataset"], # choosing sample as a covariate as we saw in the example above sample-to-sample technical variance
                              continuous_covariate_keys= ["pct_counts_mt", "total_counts"])

/Users/saidalkildani/Desktop/DS_AI/Projects/EndoGyn/venv1.venv/lib/python3.11/site-packages/scvi/data/fields/_base_field.py:63: UserWarning: adata.layers[counts] does not contain unnormalized count data. Are you sure this is what you want?
  self.validate_field(adata)


In [23]:
model = scvi.model.SCVI(adata_train, n_layers=2, n_latent=30, gene_likelihood="nb")
model.train(accelerator="gpu")

/Users/saidalkildani/Desktop/DS_AI/Projects/EndoGyn/venv1.venv/lib/python3.11/site-packages/scvi/train/_trainrunner.py:69: UserWarning: `accelerator` has been set to `mps`. Please note that not all PyTorch/Jax operations are supported with this backend. as a result, some models might be slower and less accurate than usuall. Please verify your analysis!Refer to https://github.com/pytorch/pytorch/issues/77764 for more details.
  accelerator, lightning_devices, device = parse_device_args(
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/Users/saidalkildani/Desktop/DS_AI/Projects/EndoGyn/venv1.venv/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Epoch 113/113: 100%|██████████| 113/113 [25:32<00:00, 13.53s/it, v_num=1, train_loss_step=452, train_loss_epoch=443]

`Trainer.fit` stopped: `max_epochs=113` reached.


Epoch 113/113: 100%|██████████| 113/113 [25:32<00:00, 13.56s/it, v_num=1, train_loss_step=452, train_loss_epoch=443]


In [27]:
# Get normalized counts for both sets (prevents data leakage)
norm_train = model.get_normalized_expression(
    adata_train, 
    library_size=1e4,
    return_numpy=False
)
norm_test = model.get_normalized_expression(
    adata_test,
    library_size=1e4, 
    return_numpy=False
)


INFO     Input AnnData not setup with scvi-tools. attempting to transfer AnnData setup                             


/Users/saidalkildani/Desktop/DS_AI/Projects/EndoGyn/venv1.venv/lib/python3.11/site-packages/scvi/data/fields/_base_field.py:63: UserWarning: adata.layers[counts] does not contain unnormalized count data. Are you sure this is what you want?
  self.validate_field(adata)


In [30]:
# Create DataFrames with targets
norm_train_df = norm_train.join(adata_train.obs["target"])
norm_test_df = norm_test.join(adata_test.obs["target"])

In [ ]:
# Save normalized datasets
norm_train_df.to_parquet(os.path.join(processed_data_path, "scvi_normalized_train.parquet"))
norm_test_df.to_parquet(os.path.join(processed_data_path, "scvi_normalized_test.parquet"))